In [1]:
import numpy as np
from Common_function import *
import matplotlib.pyplot as plt
from itertools import product
x = 3  #how are is the distance set
Total_E_UGV = 500000
Total_E_UAV = 15000
UAV_E_cost = 1250
UAV_E_s_cost = 2500
UGV_E_cost_with_UAV = 1000
UGV_E_cost_without_UAV = 800
Charging_speed = 1000
speed_uav =30
speed_ugv =10
survey_time = 1/6
waypoints = [(20, 10), (24, 37), (36, 43), (8, 8), (0, 50), (30, 44), (28, 4), (32, 25)]
# waypoints = [(20,10), (10,10),(15,30)]
# plan_output = "0 -> 1 -> 2 -> 0"
plan_output = "0 -> 3 -> 4 -> 1 -> 5 -> 2 -> 7 -> 6 -> 0"
max_radius = ((Total_E_UAV - UAV_E_s_cost) / UAV_E_cost) / 2
radii = np.linspace(1, max_radius, x)
ordered_points = [waypoints[int(node)] for node in plan_output.split() if node.isdigit()]
ordered_points.append(ordered_points[0])


In [2]:
def calculate_energy_remaining_inter(Current_E_UGV, Curreent_E_UAV, inter_distance_with_drone,inter_distance_without_drone,UGV_outer_path_distances_seg,UAV_path_segment, UAV_path_segment_distance):

    UGV_energy_remaining = Current_E_UGV
    UAV_energy_remaining = Curreent_E_UAV

    
    charging_distance = inter_distance_with_drone + UGV_outer_path_distances_seg

        
    UAV_Trip_Cost = UAV_path_segment_distance * UAV_E_cost + UAV_E_s_cost * (len(UAV_path_segment)-2)
    UGV_Trip_Cost = inter_distance_without_drone * UGV_E_cost_without_UAV
        
    UAV_energy_remaining -= UAV_Trip_Cost 
    UGV_energy_remaining -= UGV_Trip_Cost 
        
    if UAV_energy_remaining < Total_E_UAV:
        charge_amount = min(Charging_speed * charging_distance , UAV_Trip_Cost, UGV_energy_remaining)
        UGV_energy_remaining -= charge_amount
        UAV_energy_remaining += charge_amount
        
    if UGV_energy_remaining < 0:
        return -1 , UAV_energy_remaining
    elif UAV_energy_remaining < 0:
        return UGV_energy_remaining, -1  # Indicating mission failed for both
    print(f"UGV_Trip_Cost: {UGV_Trip_Cost:.2f},UAV_Trip_Cost: {UAV_Trip_Cost:.2f},charging_distance: {charging_distance:.2f},charge_amount: {charge_amount:.2f}")
    print(f"UGV_energy_remaining: {UGV_energy_remaining:.2f},UAV_energy_remaining: {UAV_energy_remaining:.2f} \n" )
    return UGV_energy_remaining, UAV_energy_remaining

def calculate_UAV_inter_distances_inter(UAV_path):
    UAV_inter_distances = 0
    group_distance=0
    for i in range(len(UAV_path) - 1):
        group_distance += compute_distance(UAV_path[i], UAV_path[i + 1])
    UAV_inter_distances=group_distance
    return UAV_inter_distances


In [3]:

def compute_optimized_paths_for_radius_updated_v4(ordered_points, radius_combination, speed_ugv, speed_uav, survey_time):
    UGV_outer_path = []
    UGV_path = [ordered_points[0]]
    UAV_path = []
    UGVD_inter_without_drone = []
    UGVD_inter_with_drone = []
    chord_end = None
    prev_chord_end = None
    final_wait_set = []
    radius_set = []

    for i, point in enumerate(ordered_points[:-1]):
        
        current_radius = radius_combination[i % len(radius_combination)]
        # print(radius_combination,current_radius)
        # print(i, point)
        
        # print(current_radius)
        # Exclude the start and end points from circles
        if 0 < i < len(ordered_points) - 2:
            angle = angle_between_points(ordered_points[i-1], ordered_points[i])
            nextangle = angle_between_points(ordered_points[i], ordered_points[i+1])
            chord_start = point_on_circle(ordered_points[i], angle - np.pi, current_radius)
            chord_end = point_on_circle(ordered_points[i], nextangle, current_radius)
            radius_set.append(current_radius)
            if prev_chord_end and i > 1 and circles_overlap(ordered_points[i], current_radius, ordered_points[i-1], radius_combination[(i-1) % len(radius_combination)]):
                chord_start = prev_chord_end


            final_meeting_point, final_wait_time = find_meeting_point_with_survey_final(
                ordered_points[i], current_radius, chord_start, chord_end, 
                speed_ugv, speed_uav, survey_time
            )
            final_wait_set.append(final_wait_time)
            # Update paths based on the provided structure
            if chord_start:
                UAV_path_segment = [chord_start, point, final_meeting_point]
                UAV_path.append(UAV_path_segment)
                UGV_path.append(chord_start)
                UAV_path_segment_distance = calculate_UAV_inter_distances_inter(UAV_path_segment)
                # Update UGV_outer_path as per the new structure
                if i == 1:
                    UGV_outer_path.append([ordered_points[0], chord_start])
                    Current_E_UGV = Total_E_UGV
                    Current_E_UAV = Total_E_UAV
                    UGV_outer_path_distances_seg = 0
                else:
                    UGV_outer_path.append([prev_chord_end, chord_start])
                    UGV_outer_path_distances_seg = compute_distance(prev_chord_end, chord_start)
                    Current_E_UGV = new_Current_E_UGV
                    Current_E_UAV = new_Current_E_UAV
                    
                
            if chord_end:
                UGV_path.append(chord_end)
            inter_distance_without_drone = compute_distance(chord_start, final_meeting_point)
            inter_distance_with_drone = compute_distance(final_meeting_point, chord_end)
            UGVD_inter_without_drone.append(inter_distance_without_drone)
            UGVD_inter_with_drone.append(inter_distance_with_drone)
            prev_chord_end = chord_end

            new_Current_E_UGV,new_Current_E_UAV = calculate_energy_remaining_inter(Current_E_UGV, Current_E_UAV, inter_distance_with_drone,inter_distance_without_drone,UGV_outer_path_distances_seg,UAV_path_segment, UAV_path_segment_distance)
            # print(new_Current_E_UGV,new_Current_E_UAV)
            if new_Current_E_UGV < 0 or new_Current_E_UAV < 0:
                break 
            # print(Current_E_UGV,Current_E_UAV)
            # print ( UGV_outer_path_distances, inter_distance_with_drone, UAV_distances_set, UAV_path_segment, UAV_E_cost, UAV_E_s_cost, UGVD_inter_without_drone, UGV_E_cost_without_UAV, Charging_speed)
            # UGV_energy_remaining, UAV_energy_remaining = calculate_energy_remaining_inter(Total_E_UGV, Total_E_UAV, UGV_outer_path_distances, UGVD_inter_with_drone, UAV_distances_set, UAV_path, UAV_E_cost, UAV_E_s_cost, UGVD_inter_without_drone, UGV_E_cost_without_UAV, Charging_speed)

    # Adding the last point to the paths
    UGV_path.append(ordered_points[-1])
    UGV_outer_path.append([chord_end,ordered_points[-1]])
    
    return radius_set ,UGV_path, UAV_path, UGV_outer_path, UGVD_inter_without_drone, UGVD_inter_with_drone,final_wait_set

In [5]:
def calculate_total_wait_time(final_wait_time):
    return sum(final_wait_time)
def calculate_UGV_total_distance(UGV_path, compute_distance):
    UGV_total_distance = 0
    for i in range(len(UGV_path) - 1):
        UGV_total_distance += compute_distance(UGV_path[i], UGV_path[i + 1])
    return UGV_total_distance

def calculate_UGV_outer_path_distances(UGV_outer_path):
    UGV_outer_path_distances = []
    for group in UGV_outer_path:
        UGV_outer_path_distances.append(compute_distance(group[0], group[1]))
    return UGV_outer_path_distances

def calculate_UAV_inter_distances(UAV_path):
    UAV_inter_distances = []
    for group in UAV_path:
        group_distance = 0
        for i in range(len(group) - 1):
            group_distance += compute_distance(group[i], group[i + 1])
        UAV_inter_distances.append(group_distance)
    return format_list_to_decimal(UAV_inter_distances)

UGVdistances = []
UAV_total_distances = []
UAV_distances_set = []
ordered_points = compute_ordered_points(plan_output, waypoints)
result = []
for radius_combination in product(radii, repeat=len(waypoints) - 1):
    # print(radius_set)
    radius_set, UGV_path, UAV_path, UGV_outer_path, UGVD_inter_without_drone,UGVD_inter_with_drone,final_wait_time= compute_optimized_paths_for_radius_updated_v4(ordered_points, radius_combination, speed_ugv, speed_uav, survey_time)
    
    total_wait_time = calculate_total_wait_time(final_wait_time)
    UGV_total_distance = calculate_UGV_total_distance(UGV_path, compute_distance)
    UGV_outer_path_distances = calculate_UGV_outer_path_distances(UGV_outer_path)
    
    UGV_outer_path_E_cost = [distance * UGV_E_cost_with_UAV for distance in UGV_outer_path_distances]
    total_time = total_wait_time + UGV_total_distance / speed_ugv

    
    UAV_inter_distances = calculate_UAV_inter_distances(UAV_path)
    UAV_inter_distances = [float(distance) for distance in UAV_inter_distances]
    UAV_inter_E_cost = [(distance * UAV_E_cost) + UAV_E_s_cost for distance in UAV_inter_distances]
    UAV_distances_set = calculate_UAV_distances(UAV_path, compute_distance)

    UGV_energy_remaining, UAV_energy_remaining = calculate_energy_remaining(Total_E_UGV, Total_E_UAV, UGV_outer_path_distances, UGVD_inter_with_drone, UAV_distances_set, UAV_path, UAV_E_cost, UAV_E_s_cost, UGVD_inter_without_drone, UGV_E_cost_without_UAV, Charging_speed)
    
    # print(f"radius_combination:{radius_combination}")
    if UGV_energy_remaining == -1 or UAV_energy_remaining == -1:
        
        result.append((radius_set,None))
    else:
        result.append((radius_set,round(total_radius\n")


UGV_Trip_Cost: 1144.35,UAV_Trip_Cost: 5000.00,charging_distance: 0.00,charge_amount: 0.00
UGV_energy_remaining: 498855.65,UAV_energy_remaining: 10000.00 

UGV_Trip_Cost: 685.95,UAV_Trip_Cost: 5000.00,charging_distance: 40.76,charge_amount: 5000.00
UGV_energy_remaining: 493169.70,UAV_energy_remaining: 10000.00 

UGV_Trip_Cost: 1244.82,UAV_Trip_Cost: 5000.00,charging_distance: 25.29,charge_amount: 5000.00
UGV_energy_remaining: 486924.88,UAV_energy_remaining: 10000.00 

UGV_Trip_Cost: 1393.52,UAV_Trip_Cost: 5000.00,charging_distance: 7.22,charge_amount: 5000.00
UGV_energy_remaining: 480531.35,UAV_energy_remaining: 10000.00 

UGV_Trip_Cost: 1100.69,UAV_Trip_Cost: 5000.00,charging_distance: 4.08,charge_amount: 4082.76
UGV_energy_remaining: 475347.90,UAV_energy_remaining: 9082.76 

UGV_Trip_Cost: 1599.81,UAV_Trip_Cost: 5000.00,charging_distance: 16.44,charge_amount: 5000.00
UGV_energy_remaining: 468748.08,UAV_energy_remaining: 9082.76 

UGV_Trip_Cost: 846.86,UAV_Trip_Cost: 5000.00,charging_d

In [6]:
radius_combination

(5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0)

In [7]:
x = compute_optimized_paths_for_radius_updated_v4(ordered_points, radius_combination, speed_ugv, speed_uav, survey_time)

UGV_Trip_Cost: 3430.43,UAV_Trip_Cost: 13207.88,charging_distance: 2.86,charge_amount: 2864.13
UGV_energy_remaining: 493705.44,UAV_energy_remaining: 4656.25 

UGV_Trip_Cost: 2857.44,UAV_Trip_Cost: 14671.98,charging_distance: 33.47,charge_amount: 14671.98
UGV_energy_remaining: 476176.01,UAV_energy_remaining: 4656.25 

UGV_Trip_Cost: 3556.03,UAV_Trip_Cost: 12737.34,charging_distance: 20.63,charge_amount: 12737.34
UGV_energy_remaining: 459882.64,UAV_energy_remaining: 4656.25 



In [8]:
result

[([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 14.97),
 ([1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0], 14.93),
 ([1.0, 1.0, 1.0, 1.0, 1.0, 5.0, 1.0], 14.93),
 ([1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0], 14.75),
 ([1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 1.0], 14.71),
 ([1.0, 1.0, 1.0, 1.0, 3.0, 5.0, 1.0], 14.71),
 ([1.0, 1.0, 1.0, 1.0, 5.0], 13.26),
 ([1.0, 1.0, 1.0, 1.0, 5.0], 13.26),
 ([1.0, 1.0, 1.0, 1.0, 5.0], 13.26),
 ([1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0], 14.86),
 ([1.0, 1.0, 1.0, 3.0, 1.0, 3.0, 1.0], 14.82),
 ([1.0, 1.0, 1.0, 3.0, 1.0, 5.0, 1.0], 14.82),
 ([1.0, 1.0, 1.0, 3.0, 3.0, 1.0, 1.0], 14.64),
 ([1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 1.0], 14.6),
 ([1.0, 1.0, 1.0, 3.0, 3.0, 5.0, 1.0], 14.6),
 ([1.0, 1.0, 1.0, 3.0, 5.0], 13.22),
 ([1.0, 1.0, 1.0, 3.0, 5.0], 13.22),
 ([1.0, 1.0, 1.0, 3.0, 5.0], 13.22),
 ([1.0, 1.0, 1.0, 5.0, 1.0, 1.0, 1.0], 14.8),
 ([1.0, 1.0, 1.0, 5.0, 1.0, 3.0, 1.0], 14.77),
 ([1.0, 1.0, 1.0, 5.0, 1.0, 5.0, 1.0], 14.77),
 ([1.0, 1.0, 1.0, 5.0, 3.0], 13.3),
 ([1.0, 1.0, 1.0, 5.0, 3.0], 13.3),
 ([1